In [55]:
import pandas as pd
import pickle

In [3]:
sample_code_dic = {"01": "Primary Solid Tumor",
                   "11": "Solid Tissue Normal",
                   "02": "Recurrent Solid Tumor",
                   "06": "Metastatic",
                   "07": "Additional Metastatic",
                   "05": "Additional - New Primary",
                   "03": "Primary Blood Derived Cancer - Peripheral Blood"}

In [8]:
df = pd.read_csv("data/RNASeq_barcodes.txt", delimiter="\t")
sample_codes=list(i.split("-")[3][:-1] for i in df.sample_barcode)
stats = {sample_code_dic[x]:sample_codes.count(x) for x in sample_codes}

In [9]:
stats

{'Additional - New Primary': 11,
 'Additional Metastatic': 1,
 'Metastatic': 395,
 'Primary Blood Derived Cancer - Peripheral Blood': 173,
 'Primary Solid Tumor': 9706,
 'Recurrent Solid Tumor': 46,
 'Solid Tissue Normal': 737}

In [73]:
tumor_barcodes = [i for i in df.sample_barcode if i.split("-")[3][:-1]=="01"]
normal_barcodes = [i for i in df.sample_barcode if i.split("-")[3][:-1]=="11"]
tumor_persons = set([i.split("-")[2] for i in tumor_barcodes])
normal_persons = set([i.split("-")[2] for i in normal_barcodes])
overlap = normal_persons.intersection(tumor_persons)
overlap_barcodes = [i for i in df.sample_barcode if 
                    (i.split("-")[2] in overlap and i.split("-")[3][:-1] in ["01", "11"])]

In [56]:
pickle.dump(overlap, open("data/matched_tumor_normal_patients.pkl", "wb"))

In [63]:
mRNA = pd.read_csv("data/RNASeq_transposed.tsv", delimiter="\t")

In [95]:
tumor_normal_df = mRNA[mRNA.sample_barcode.isin(overlap_barcodes)].copy()
tumor_normal_df["label"] = [1 if i.split("-")[3][:-1] == "11" else 0 
                            for i in tumor_normal_df.sample_barcode]
tumor_normal_df["patient_id"] = [i.split("-")[2] 
                                 for i in tumor_normal_df.sample_barcode]
final_df = tumor_normal_df.dropna(axis=1)

In [ ]:
final_df.to_csv("data/RNASeq_tumor_v_normal.tsv", sep="\t", index=None)